In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from pathlib import Path
import pandas as pd
import data_transformation as transformer
import crewai_mapper as mapper
import data_ingestion as ingestion

In [ ]:
INPUT_PATH = Path("./input_data/Openstaande posten crediteuren 300423 SAP.XLSX")
OUTPUT_PATH = Path("./output_data")

In [ ]:
VALIDATION_RULES = ingestion.VALIDATION_RULES

In [ ]:
df = ingestion.file_ingestion(path_dir=INPUT_PATH)

In [ ]:
sub_df = transformer.transform_nonstandard_SAP(df=df, new_column_name="Cust. name")
csv =sub_df.head(10).to_csv(sep=',')

In [ ]:
DESTINATION_COLUMNS = VALIDATION_RULES["parametres"]["names"]
SOURCE_COLUMNS = sub_df.columns.to_list()
TO_DISCARD = VALIDATION_RULES["parametres"]["to_discard"]

In [ ]:
response = mapper.get_mapping(expected=DESTINATION_COLUMNS, input=csv)

In [ ]:
source_columns = transformer.extract_substrings(response, SOURCE_COLUMNS)
destination_columns = transformer.extract_substrings(response, DESTINATION_COLUMNS)

In [ ]:
source = list(dict.fromkeys(source_columns))
dest = list(dict.fromkeys(destination_columns))

In [ ]:
source

In [ ]:
dest

In [ ]:
mapping = dict(zip(source, dest))

In [ ]:
mapped_df = transformer.mapper(mapping=mapping, to_discard=TO_DISCARD, df=df)

In [ ]:
try:
    df = df.rename(columns=mapping)
    for col in TO_DISCARD:
        try:
            df = df.drop(columns=[col])
        except KeyError:
            print(f"Column '{col}' not found in the DataFrame and will be skipped.")
except KeyError as e:
    print(f"Error: {e}")
    print("Please check the column names in the mapping dictionary and to_discard list.")

In [ ]:
print(mapped_df)